In [2]:
import sys
import os
from sentence_transformers import CrossEncoder
import logging

sub_s = os.sep.join(os.getcwd().split(os.sep)[:-1] + ['submodule', 'financerag'])
sys.path.append(sub_s)

from financerag.rerank import CrossEncoderReranker
from financerag.retrieval import DenseRetrieval, SentenceTransformerEncoder
from financerag.tasks import FinDER
logging.basicConfig(level=logging.INFO)

In [3]:
finder_task = FinDER()

INFO:financerag.common.loader:Loading Corpus...
INFO:financerag.common.loader:Loaded 13867 Documents.
INFO:financerag.common.loader:Corpus Example: {'id': 'ADBE20230004', 'title': 'ADBE OVERVIEW', 'text': 'Adobe is a global technology company with a mission to change the world through personalized digital experiences. For over four decades, Adobe’s innovations have transformed how individuals, teams, businesses, enterprises, institutions, and governments engage and interact across all types of media. Our products, services and solutions are used around the world to imagine, create, manage, deliver, measure, optimize and engage with content across surfaces and fuel digital experiences. We have a diverse user base that includes consumers, communicators, creative professionals, developers, students, small and medium businesses and enterprises. We are also empowering creators by putting the power of artificial intelligence (“AI”) in their hands, and doing so in ways we believe are responsi

In [4]:
encoder_model = SentenceTransformerEncoder(
    model_name_or_path='intfloat/e5-large-v2',
    query_prompt='query: ',
    doc_prompt='passage: ',
)

retrieval_model = DenseRetrieval(
    model=encoder_model
)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/e5-large-v2
c:\Users\tenet\anaconda3\envs\financeRAG_acm\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tenet\.cache\huggingface\hub\models--intfloat--e5-large-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/window

In [5]:
retrieval_model = DenseRetrieval(
    model=encoder_model
)

retrieval_result = finder_task.retrieve(
    retriever=retrieval_model
)

INFO:financerag.retrieval.dense:Encoding queries...
Batches: 100%|██████████| 4/4 [00:01<00:00,  3.69it/s]
INFO:financerag.retrieval.dense:Sorting corpus by document length...
INFO:financerag.retrieval.dense:Encoding corpus in batches... This may take a while.
INFO:financerag.retrieval.dense:Encoding batch 1/1...
Batches: 100%|██████████| 217/217 [07:31<00:00,  2.08s/it]


In [6]:
# Print a portion of the retrieval results to verify the output.
print(f"Retrieved results for {len(retrieval_result)} queries. Here's an example of the top 5 documents for the first query:")

for q_id, result in retrieval_result.items():
    print(f"\nQuery ID: {q_id}")
    # Sort the result to print the top 5 document ID and its score
    sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)

    for i, (doc_id, score) in enumerate(sorted_results[:5]):
        print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

    break  # Only show the first query

Retrieved results for 216 queries. Here's an example of the top 5 documents for the first query:

Query ID: q00001
  Document 1: Document ID = MSFT20230966, Score = 0.8739960789680481
  Document 2: Document ID = MSFT20230216, Score = 0.8645689487457275
  Document 3: Document ID = MSFT20230015, Score = 0.859443724155426
  Document 4: Document ID = MSFT20230254, Score = 0.8580108284950256
  Document 5: Document ID = MSFT20230155, Score = 0.8534095287322998


In [7]:
reranker = CrossEncoderReranker(
    model=CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
)

c:\Users\tenet\anaconda3\envs\financeRAG_acm\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tenet\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: cuda


In [8]:
reranking_result = finder_task.rerank(
    reranker=reranker,
    results=retrieval_result,
    top_k=100,  # Rerank the top 100 documents
    batch_size=32
)

INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-100....
Batches: 100%|██████████| 675/675 [02:31<00:00,  4.46it/s]


In [9]:
# Print a portion of the reranking results to verify the output.
print(f"Reranking results for {len(reranking_result)} queries. Here's an example of the top 5 documents for the first query:")

for q_id, result in reranking_result.items():
    print(f"\nQuery ID: {q_id}")
    # Sort the result to print the top 5 document ID and its score
    sorted_results = sorted(result.items(), key=lambda x: x[1], reverse=True)

    for i, (doc_id, score) in enumerate(sorted_results[:5]):
        print(f"  Document {i + 1}: Document ID = {doc_id}, Score = {score}")

    break  # Only show the first query

Reranking results for 216 queries. Here's an example of the top 5 documents for the first query:

Query ID: q00001
  Document 1: Document ID = MSFT20230254, Score = 6.622737884521484
  Document 2: Document ID = MSFT20230233, Score = 6.088945388793945
  Document 3: Document ID = MSFT20230230, Score = 5.898367881774902
  Document 4: Document ID = MSFT20230236, Score = 5.747088432312012
  Document 5: Document ID = MSFT20230216, Score = 5.488572120666504


In [10]:
# Step 7: Save results
# -------------------
# Save the results to the specified output directory as a CSV file.
output_dir = './results'
finder_task.save_results(output_dir=output_dir)

# Confirm the results have been saved.
print(f"Results have been saved to {output_dir}/FinDER/results.csv")

INFO:financerag.tasks.BaseTask:Output directory set to: ./results\FinDER
INFO:financerag.tasks.BaseTask:Saving top 10 results to CSV file: ./results\FinDER\results.csv
INFO:financerag.tasks.BaseTask:Writing header ['query_id', 'corpus_id'] to CSV.
INFO:financerag.tasks.BaseTask:Top 10 results saved successfully to ./results\FinDER\results.csv


Results have been saved to ./results/FinDER/results.csv


In [15]:
import pandas as pd
from financerag.tasks import FinDER
df = pd.read_csv('../files/icaif-24-finance-rag-challenge/FinDER_qrels.tsv', sep='\t')
qrels_dict = df.groupby('query_id').apply(lambda x: dict(zip(x['corpus_id'], x['score']))).to_dict()


C:\Users\tenet\AppData\Local\Temp\ipykernel_5876\50549677.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  qrels_dict = df.groupby('query_id').apply(lambda x: dict(zip(x['corpus_id'], x['score']))).to_dict()


In [18]:
finder_task.evaluate(qrels_dict, reranking_result, [1, 5, 10])

INFO:financerag.tasks.BaseTask:For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:NDCG@1: 0.2500
INFO:financerag.tasks.BaseTask:NDCG@5: 0.3363
INFO:financerag.tasks.BaseTask:NDCG@10: 0.3701
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:MAP@1: 0.2188
INFO:financerag.tasks.BaseTask:MAP@5: 0.3051
INFO:financerag.tasks.BaseTask:MAP@10: 0.3217
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:Recall@1: 0.2188
INFO:financerag.tasks.BaseTask:Recall@5: 0.3969
INFO:financerag.tasks.BaseTask:Recall@10: 0.4865
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:P@1: 0.2500
INFO:financerag.tasks.BaseTask:P@5: 0.1125
INFO:financerag.tasks.BaseTask:P@10: 0.0719


({'NDCG@1': 0.25, 'NDCG@5': 0.3363, 'NDCG@10': 0.37007},
 {'MAP@1': 0.21875, 'MAP@5': 0.30514, 'MAP@10': 0.3217},
 {'Recall@1': 0.21875, 'Recall@5': 0.39687, 'Recall@10': 0.48646},
 {'P@1': 0.25, 'P@5': 0.1125, 'P@10': 0.07188})